In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np


# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
encoder = OneHotEncoder(sparse=False)
time_encoded = encoder.fit_transform(train_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
train_data = pd.concat([train_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['ID', '풍속 (m/s)'], axis=1)  # 입력 변수들
y_train = train_data['풍속 (m/s)']  # 출력 변수 (풍속)

# HyperOpt 최적화 함수 정의
def objective(params):
    model = ExtraTreesRegressor(**params, random_state=42)
    mae_scores = -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    return {'loss': np.mean(mae_scores), 'status': STATUS_OK}

# HyperOpt 탐색 공간 정의
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),
    'max_depth': hp.quniform('max_depth', 10, 100, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
    'max_features': hp.uniform('max_features', 0.1, 1)
}

# HyperOpt 최적화 수행
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# 최적의 하이퍼파라미터로 모델 생성
best_params = {
    'n_estimators': int(best['n_estimators']),
    'max_depth': int(best['max_depth']),
    'min_samples_split': best['min_samples_split'],
    'min_samples_leaf': best['min_samples_leaf'],
    'max_features': best['max_features']
}
model = ExtraTreesRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')

# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
time_encoded = encoder.transform(test_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
test_data = pd.concat([test_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 테스트 데이터로 풍속 예측을 수행합니다.
X_test = test_data.drop('ID', axis=1)  # 테스트 입력 변수들
y_pred = model.predict(X_test)  # 테스트 데이터로 풍속 예측

# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')
submission['풍속 (m/s)'] = y_pred

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")

# Cross Validation을 활용하여 평균 MAE 계산 (HyperOpt를 거친 모델에 대해 재계산)
model = ExtraTreesRegressor(**best_params, random_state=42)
mae_scores = -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
print("평균 MAE:", np.mean(mae_scores))


  2%|█▎                                                                | 2/100 [00:00<00:05, 16.53trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".f

  4%|██▋                                                               | 4/100 [00:00<00:05, 17.45trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

  8%|█████▎                                                            | 8/100 [00:00<00:05, 16.91trial/s, best loss=?]

...

 97%|███████████████████████████████████████████████████████████████  | 97/100 [00:05<00:00, 19.94trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

100%|████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.01trial/s, best loss=?]


C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

AllTrialsFailed: 